In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
# add our custom modules
import sys
sys.path.append("../src") # go to parent dir
from microgrid_env import MicrogridEnv

In [10]:
env_df = pd.read_csv("../data/cropped.csv")
# names consistent with `update_environment`
env_df = env_df.rename(
  columns={"Wind Speed": "wind_speed",
           "Solar Irradiance": "solar_irradiance",
           "Grid Electricity Price": "rate_consumption_charge"},
)
# compute total energy demand from example houses
env_df["energy_demand"] = env_df["warehouse 1"] + env_df["small hotel 1"]
env_df = env_df.drop(columns=["warehouse 1", "small hotel 1"])

In [17]:
env_df

,wind_speed,solar_irradiance,rate_consumption_charge,energy_demand
0,45.900,0.1592,0.06,57.790509
1,46.872,0.0850,0.06,52.585363
2,47.340,0.0000,0.06,51.481020
3,40.608,0.0000,0.06,53.320262
4,42.732,0.0000,0.06,56.951506
...,...,...,...,...
4314,6.480,67.7188,0.19,150.229071
4315,9.036,0.9450,0.19,142.505024
4316,9.828,0.0695,0.06,111.824389
4317,2.592,0.1174,0.06,84.980106


In [18]:
data_dict = env_df.to_dict(orient="list")

In [19]:
env = MicrogridEnv(data_dict)

In [25]:
env.action_space.seed(42)
observation, info = env.reset(seed=42)

for _ in range(10000):
    action = env.action_space.sample() # TODO: this should probably come from our model
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()